In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2023-11-08 20:45:54.505879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pip install tensorflow_datasets

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow_datasets as tfds

In [4]:
(train, test) = tfds.load('mnist',
                                split = ['train','test'])

In [5]:
train

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [6]:
test

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

create dataframe

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [17]:
def load_dataset(train_size, test_size):
    train, test = tf.keras.datasets.mnist.load_data()

    # take a sample
    train_idx = np.random.randint(low=0, high=train[0].shape[0], size=train_size)
    test_idx = np.random.randint(low=0, high=test[0].shape[0], size=test_size)
    X_train = train[0][train_idx].reshape(-1,28*28)
    y_train = train[1][train_idx].reshape(-1,1)
    X_test = test[0][test_idx].reshape(-1,28*28)
    y_test = test[1][test_idx].reshape(-1,1)
    
    scale = StandardScaler()
    X_train = scale.fit_transform(X_train)
    X_test = scale.transform(X_test)

    OH = OneHotEncoder(categories='auto', sparse=False)
    y_train = OH.fit_transform(y_train)
    y_test = OH.transform(y_test)
    print('X_train:',X_train.shape)
    print('y_train:',y_train.shape)
    print('X_test:',X_test.shape)
    print('y_test:',y_test.shape)
    print('Min:', X_train.min())
    print('Max:', X_train.max())

    return X_train, X_test, y_train, y_test

In [26]:
X_train, X_test, y_train, y_test = load_dataset(60000,10000)

X_train: (60000, 784)
y_train: (60000, 10)
X_test: (10000, 784)
y_test: (10000, 10)
Min: -1.276624781734497
Max: 244.9469330287314


Optimizing at the initialization stage

In [27]:
from tensorflow.keras import layers

In [28]:
init = tf.initializers.zeros()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [29]:
def simple_model(initializer, activation='relu'):
    return tf.keras.Sequential([layers.Dense(32, activation, input_shape=(784,), kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(10, activation = 'softmax', kernel_initializer=tf.keras.initializers.glorot_normal())



    ])

In [22]:
model_zeros = simple_model(init, activate)

In [31]:
model_zeros.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_zeros.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

Epoch 1/10
19/19 [==============================] - 3s 62ms/step - loss: 2.3026 - accuracy: 0.1091 - val_loss: 2.3025 - val_accuracy: 0.1142
Epoch 2/10
19/19 [==============================] - 1s 41ms/step - loss: 2.3025 - accuracy: 0.1109 - val_loss: 2.3025 - val_accuracy: 0.1142
Epoch 3/10
19/19 [==============================] - 1s 41ms/step - loss: 2.3025 - accuracy: 0.1109 - val_loss: 2.3024 - val_accuracy: 0.1142
Epoch 4/10
19/19 [==============================] - 1s 46ms/step - loss: 2.3024 - accuracy: 0.1109 - val_loss: 2.3023 - val_accuracy: 0.1142
Epoch 5/10
19/19 [==============================] - 1s 36ms/step - loss: 2.3024 - accuracy: 0.1109 - val_loss: 2.3023 - val_accuracy: 0.1142
Epoch 6/10
19/19 [==============================] - 1s 35ms/step - loss: 2.3023 - accuracy: 0.1109 - val_loss: 2.3022 - val_accuracy: 0.1142
Epoch 7/10
19/19 [==============================] - 1s 37ms/step - loss: 2.3023 - accuracy: 0.1109 - val_loss: 2.3022 - val_accuracy: 0.1142
Epoch 8/10
19

In [32]:
init = tf.initializers.random_normal()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_normal = simple_model(init, activate)
model_normal.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_normal.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

/opt/anaconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/10
19/19 [==============================] - 2s 56ms/step - loss: 2.3022 - accuracy: 0.2097 - val_loss: 2.3015 - val_accuracy: 0.2083
Epoch 2/10
19/19 [==============================] - 1s 33ms/step - loss: 2.3016 - accuracy: 0.1956 - val_loss: 2.3009 - val_accuracy: 0.1927
Epoch 3/10
19/19 [==============================] - 1s 34ms/step - loss: 2.3011 - accuracy: 0.1843 - val_loss: 2.3005 - val_accuracy: 0.1826
Epoch 4/10
19/19 [==============================] - 1s 35ms/step - loss: 2.3006 - accuracy: 0.1774 - val_loss: 2.3000 - val_accuracy: 0.1755
Epoch 5/10
19/19 [==============================] - 1s 40ms/step - loss: 2.3003 - accuracy: 0.1687 - val_loss: 2.2997 - val_accuracy: 0.1681
Epoch 6/10
19/19 [==============================] - 1s 36ms/step - loss: 2.2999 - accuracy: 0.1654 - val_loss: 2.2993 - val_accuracy: 0.1620
Epoch 7/10
19/19 [==============================] - 1s 43ms/step - loss: 2.2996 - accuracy: 0.1594 - val_loss: 2.2989 - val_accuracy: 0.1575
Epoch 8/10
19

In [33]:
init = tf.initializers.he_uniform()
activate = 'relu'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model_heu = simple_model(init, activate)
model_heu.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_heu.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data=(X_test, y_test))

/opt/anaconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer HeUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/10
19/19 [==============================] - 3s 60ms/step - loss: 2.5303 - accuracy: 0.1597 - val_loss: 2.2306 - val_accuracy: 0.2193
Epoch 2/10
19/19 [==============================] - 1s 33ms/step - loss: 2.0716 - accuracy: 0.2975 - val_loss: 1.9314 - val_accuracy: 0.3693
Epoch 3/10
19/19 [==============================] - 1s 47ms/step - loss: 1.8134 - accuracy: 0.4204 - val_loss: 1.7113 - val_accuracy: 0.4684
Epoch 4/10
19/19 [==============================] - 1s 36ms/step - loss: 1.6008 - accuracy: 0.5142 - val_loss: 1.5153 - val_accuracy: 0.5552
Epoch 5/10
19/19 [==============================] - 1s 36ms/step - loss: 1.4101 - accuracy: 0.5882 - val_loss: 1.3404 - val_accuracy: 0.6148
Epoch 6/10
19/19 [==============================] - 1s 36ms/step - loss: 1.2463 - accuracy: 0.6411 - val_loss: 1.1942 - val_accuracy: 0.6621
Epoch 7/10
19/19 [==============================] - 1s 35ms/step - loss: 1.1118 - accuracy: 0.6815 - val_loss: 1.0764 - val_accuracy: 0.6948
Epoch 8/10
19

In [34]:
for i in [model_heu, model_normal, model_glorotu, model_zeros]:
    plt.plot(i.history.history['val_accuracy'], label=i.get_config()['layers'][1]['config']['kernel_initializer']['class_name'])

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')

NameError: name 'model_glorotu' is not defined

In [ ]:
def simple_model_bn(initializer = 'he_uniform', activation='relu'):
    return tf.keras.Sequential([layers.Dense(32, activation, input_shape=(784,), kernel_initializer=initializer),
                                layers.BatchNormalization(trainable=True, scale=True, center=True),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.BatchNormalization(trainable=True, scale=True, center=True),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.BatchNormalization(trainable=True, scale=True, center=True),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.BatchNormalization(trainable=True, scale=True, center=True),
                                layers.Dense(10, activation = 'softmax', kernel_initializer=tf.keras.initializers.glorot_normal())



    ])

In [ ]:
simple_model_bn().summary()

In [ ]:
init = tf.keras.initializers.random_normal()
activate = 'relu'

optimizer = tf.keras.optimizers.SGD(0.01)
model_bn = simple_model_bn(init, activate)
model_bn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model_bn.fit(X_train, y_train, epochs=10, batch_size=3200, validation_data = (X_test, y_test))